<a href="https://colab.research.google.com/github/Kyveli-tsioli/hello-world/blob/main/Copy_of_11_01_%E2%80%93_Automatic_differentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation cell
%%shell
if ! command -v julia 2>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.0/julia-1.0.5-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'

In [ ]:
using Pkg

Pkg.add(Pkg.PackageSpec(;name="Flux", version=v"0.9.0"))

pkg"add Images"
pkg"add Metalhead"

pkg"precompile"

using Flux
using Images
using Metalhead




#automatic differentiation
#each layer really depends on that layer inputs and that layers outputs
#error term backprogate gradually through the network
#conclusion: how about i implement a library for this?
#abstract over all the mathematical calculations and build a for loop
#useful abstraction
#gpu cpu neural networks

# Automatic differentiation #

While it is obvious when you start working with a multi-layer perceptron that each layer’s output only depends on it’s inputs, you soon also realise from your maths and code that the gradient with respect to its weights only depend on its input and backpropagated error – making it possible to easily implement layer in isolation. So, why not build a library or framework? You are not alone in thinking this way.

What do frameworks/libraries achieve?

* Increase productivity
* Enable hardware acceleration and scaling (more easily)
* Lowers barrier to entry
* Encapsulate “best practices”
* Code sharing (becoming a lungua franca)

Some potential bad things:

* Limiting expressivity
* “Technical debt”
* Computational overhead

Most frameworks/libraries that you encounter really do two things:

1. Automatic differentation
2. GPU/TPU offloading

We will primarily touch upon the former, although the latter is interesting in its own right, it belongs in a high-performance computing class (besides, as much as I like low-level coding, I am actually not that good at the inner workings of GPUs/TPUs).

## History time ##



### SN/Lush by LeCun and Bottou (1987) ###

[Lush](http://lush.sourceforge.net/) oldest one that I am aware of, with a release as recent as 2011!

* Object-oriented Lisp
* Layer objects, that are chained together
* Generates C code
* Used for the original LeNet!

LeNet code example:

```lisp
(de new-lenet5 (image-height 
                image-width
                ki0 kj0 si0 sj0 ki1 kj1 si1 sj1
                hid output-size net-param)
  (let ((table0 (full-table 1 6))
        (table1 (int-array 60 2))
        (table2 (full-table 16 hid)))
    (table1 ()()
            '(0 0  1 0  2 0
                1 1  2 1  3 1
                2 2  3 2  4 2
                3 3  4 3  5 3
                4 4  5 4  0 4
                5 5  0 5  1 5

                0 6  1 6  2 6  3 6
                1 7  2 7  3 7  4 7
                2 8  3 8  4 8  5 8
                3 9  4 9  5 9  0 9
                4 10  5 10  0 10  1 10
                5 11  0 11  1 11  2 11

                0 12  1 12   3 12  4 12
                1 13  2 13   4 13  5 13
                2 14  3 14   5 14  0 14

                0 15  1 15  2 15  3 15  4 15  5 15 ))
    (new net-cscscf
         image-height image-width
         ki0 kj0 table0 si0 sj0
         ki1 kj1 table1 si1 sj1
         ;; WARNING: those two numbers must be changed 
         ;; when image-height/image-width change
         (/ (- (/ (- image-height (1- ki0)) si0) (1- ki1)) si1)
         (/ (- (/ (- image-width (1- kj0)) sj0) (1- kj1)) sj1)
         (full-table 16 hid)
         output-size
         net-param)))
```

#LISP
#LeNet: CNN
lisp^


### Theano by Breuleux et al. (2008) ###

[Theano](http://www.deeplearning.net/software/theano/) was (abandoned since 2017) Developed at the [Mila group in Québec](https://mila.quebec/en/), it is the first one that I know of that takes an algebraic approach.

* Python-based
* Builds a computation graph out of mathematical objects
* Generated C and CUDA code

[LeNet code example](https://github.com/suriyadeepan/theano/blob/master/code/04CNN/lenet.py):

```python
x = T.matrix('x')
y = T.ivector('y')

# convert input x to form (batch_size,1,28,28)
layer0_input = x.reshape((batch_size,1,28,28))

# setup random stream
rng = np.random.RandomState(123455)

# build layer0
layer0 = ConvPoolLayer(rng=rng,input=layer0_input,
                      image_shape=(batch_size,1,28,28),
                      filter_shape=(20,1,5,5))

## Layer 1 setup ##
layer1 = ConvPoolLayer(rng=rng,input=layer0.output,
                      image_shape=(batch_size,20,12,12),
                      filter_shape=(50,20,5,5))

## Layer 2 : Hidden Layer setup ##
# layer1 output shape : batch_sizex50x4x4
# layer2_h input shape req : batch_size x (50*4*4)
layer2_h_input = layer1.output.flatten(2)
# n_in = 50x4x4 pixels; n_out = 500 hidden nodes
layer2_h = HiddenLayer(rng=rng,input=layer2_h_input,n_in=50*4*4,n_out=500)

# Layer 3 : Output layer : LogisticRegression
layer3_o = LogisticRegression(input=layer2_h.output,n_in=500,n_out=10)
```

### Torch by Collobert et al. (2008) ###

[Lua](http://torch.ch/) wrapper around a very clean C library. It is notable that there is overlap in personell between Torch and Lush.

* Layers and high-level operations defined in Lua
* No code generation, simply calls C/CUDA through its C API
* Enjoyed widespread use at Facebook, DeepMind, etc. up until very recently

[LeNet code example](https://github.com/gwgundersen/lenet5/blob/master/lenet.lua):

```lua
function lenet.new(geometry)

    local nChannels = geometry[1]  -- B&W=1, RGB=3.
    local imgWidth = geometry[2]
    local imgHeight = geometry[3]

    local model = nn.Sequential()

    -----------------------------------
    -- 2 ConvNet layers
    -- Arguments:
    --      input depth, output depth, kernel width, kernel height
    --      [step width], [step height], [padding width], [padding height]
    model:add(nn.SpatialConvolution(nChannels, N_FEATURE_MAPS_1ST_L,
        FILTER_SIZE, FILTER_SIZE, STEP_SIZE, STEP_SIZE, PADDING, PADDING))
    model:add(nn.SpatialMaxPooling(POOL_SIZE, POOL_SIZE, STEP_SIZE, STEP_SIZE))
    model:add(nn.ReLU())
    model:add(nn.SpatialConvolution(N_FEATURE_MAPS_1ST_L, N_FEATURE_MAPS_2ND_L,
        FILTER_SIZE, FILTER_SIZE, STEP_SIZE, STEP_SIZE, PADDING, PADDING))
    model:add(nn.SpatialMaxPooling(POOL_SIZE, POOL_SIZE, STEP_SIZE, STEP_SIZE))
    model:add(nn.ReLU())

    -- Programmatically compute output size.
    local dummy_input  = torch.Tensor(nChannels, imgWidth, imgHeight)
    model:forward(dummy_input)
    local output_w = model.modules[6].output[1]:size()[1]  -- Convert tensor to value.
    local output_h = model.modules[6].output[2]:size()[1]
    print('<lenet> output (w, h):', output_w, output_h)

    -----------------------------------
    -- 1 fully-connected layer
    local newShape = N_FEATURE_MAPS_2ND_L * output_w * output_h
    model:add(nn.Reshape(newShape))
    model:add(nn.Linear(newShape, WIDTH_FC_L))
    model:add(nn.ReLU())

    -----------------------------------
    -- Output layer
    model:add(nn.Linear(WIDTH_FC_L, N_CLASSES))

    return model
end
```

collobert: deep learning revolution


### Enter the hype… ###

An the “Deep Learning revolution” took off, the number of frameworks/libraries exploded:

* DeCaf/Caffe (2013)
* ApolloCaffe (2015)
* Dali (2015)
* Knet.jl (2015)
* cnn/DyNet (2015)
* Chainer (2015)
* Hype (2015)
* And more…

These are all pretty similar. Heck, I myself made [one back in 2013 with a focus on composition in language](https://github.com/ninjin/nerv).

However, noteable was that [cnn/DyNet](http://dynet.io/) introduced the concept of “dynamic auto-batching” and [Chainer](https://chainer.org/) introduced the term “define-by-run” (also referred to as “eager” execution).

### TensorFlow by Google (2015) ###

[TensorFlow](https://www.tensorflow.org/), Python bindings (mainly) for a low-level computation graph implemented in C++. 

* Allows for models to be developed in Python and then deployed on other devices.
* No code generation out of the box, but can be achieved using XLA that operates on the computation graph.
* Pushed hard inside of Google Brain and DeepMind – completely replaced Torch inside of DeepMind in about a year.
* Pretty much an industry standard, even if reseachers largely have been migrating since 2017 or so.

It had the nickname “TensorSlow” (`import tensorflow as tensorslow`) for the first year or so, but it picked up speed once it got more powerful primitives to express more dynamic inputs.

[LeNet code example](https://github.com/tensorflow/models/blob/master/research/slim/nets/lenet.py):

```python
def lenet(images, num_classes=10, is_training=False,
          dropout_keep_prob=0.5,
          prediction_fn=slim.softmax,
          scope='LeNet'):
  end_points = {}

  with tf.variable_scope(scope, 'LeNet', [images]):
    net = end_points['conv1'] = slim.conv2d(images, 32, [5, 5], scope='conv1')
    net = end_points['pool1'] = slim.max_pool2d(net, [2, 2], 2, scope='pool1')
    net = end_points['conv2'] = slim.conv2d(net, 64, [5, 5], scope='conv2')
    net = end_points['pool2'] = slim.max_pool2d(net, [2, 2], 2, scope='pool2')
    net = slim.flatten(net)
    end_points['Flatten'] = net

    net = end_points['fc3'] = slim.fully_connected(net, 1024, scope='fc3')
    if not num_classes:
      return net, end_points
    net = end_points['dropout3'] = slim.dropout(
        net, dropout_keep_prob, is_training=is_training, scope='dropout3')
    logits = end_points['Logits'] = slim.fully_connected(
        net, num_classes, activation_fn=None, scope='fc4')

  end_points['Predictions'] = prediction_fn(logits, scope='Predictions')

  return logits, end_points
```

### PyTorch by Paszke et al. (2016) ###

[PyTorch](http://pytorch.org/) is pretty much identical to Torch, but introduces a Python API.

* The Torch leadership was (still?) heavily against any non-Lua approach, so PyTorch was made by a single intern at Facebook AI Research.
* Has become the defacto standard in research as of 2018/2019, why do you think this is the case?

[LeNet code example](https://github.com/kuangliu/pytorch-cifar/blob/master/models/lenet.py):

```python
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
```

## High-level categorisation ##

Where does automatic differentiation take place?

* Abstract layers
* Computational graph
* Step-by-step on the level of the programming language

There is an ongoing “battle” as to what the “best” level to operate on is. Here are some key recent developments:

* JAX (Google, 2018), traces NumPy-esque Python code, turns into into XLA (a compiler for TensorFlow graphs), and turns it into code.
* PyTorch has made a similar tracer public earlier this year.
* Zygote.jl, also traces programmes, but does so on top of the programming language itself rather than a higher level abstraction.
* Swift for TensorFlow, akin to what Zygote.jl achieves, but does so “statically” by the virue of Swift being a statically compiled programming language.

But, why should we care as researchers and practitioners?

Remember?:

* Limiting expressivity
* “Technical debt”
* Computational overhead

The way I see it, this is a wonderful time to work with Deep Learning as the number of clever engineers that are trying to solve “our issues” is simply enormous.

## Flux.jl ##

[Flux](https://github.com/FluxML/Flux.jl) is the “standard” library for automatic differentiation in Julia, but there are others such as [TensorFlow.jl](https://github.com/malmaud/TensorFlow.jl) and [Knet.jl](https://github.com/denizyuret/Knet.jl). In relation to what we have talked about so far, it is most closely related to PyTorch, but due to some Julia magic it can be hooked up with Zygote.jl to do language-level tracing (this is in beta though) and due to how Julia interacts with LLVM, it can generate GPU/TPU code – unlike PyTorch, as far as I know.

## Logistic regression ##

Let us consider a very simple model, how do we express it in Flux?

In [ ]:
logistic(x) = 1/(1 + exp(-x))

# Assuming our “toy” dataset.
logreg = Dense(2, 1, logistic)

logreg([0.17, 0.4711])

Tracked 1-element Array{Float32,1}:
 0.5332073f0

Okay, that was cheating, let us “spell it out”:

In [ ]:
W    = param(randn(2)/100)
b    = param(0)
f(x) = logistic(W'x + b)

f([0.17, 0.4711])

0.49982126328092474 (tracked)

In [ ]:
using Flux.Tracker: gradient
gradient(f, [0.17, 0.4711])

([-0.000168639, -0.000318548] (tracked),)

Okay, so what is this `param` and “tracked” mumbo jumbo?

From [Tracker.jl](https://github.com/FluxML/Tracker.jl):

1. [array.jl](https://github.com/FluxML/Tracker.jl/blob/master/src/lib/array.jl)
2. [real.jl](https://github.com/FluxML/Tracker.jl/blob/master/src/lib/real.jl)

In [ ]:
struct TrackedArray{T,N,A<:AbstractArray{T,N}} <: AbstractArray{T,N}
  tracker::Tracked{A}
  data::A
  grad::A
  TrackedArray{T,N,A}(t::Tracked{A}, data::A) where {T,N,A} = new(t, data)
  TrackedArray{T,N,A}(t::Tracked{A}, data::A, grad::A) where {T,N,A} = new(t, data, grad)
end

In [ ]:
mutable struct TrackedReal{T<:Real} <: Real
  data::T
  tracker::Tracked{T}
end

### A minor case study, making `maximum` and `minimum` differentiable ###

Back in 2018 I was playing around with Flux, but noticed that there was no support for [`maximum`](https://docs.julialang.org/en/v1.0/base/collections/#Base.maximum) and [`minimum`](https://docs.julialang.org/en/v1.0/base/collections/#Base.minimum) that I needed for an embedding-based baseline model. So, [I added support for it and filed a pull requesh](https://github.com/FluxML/Flux.jl/pull/249) and I think it makes for an interesting example of how these libraries work.

So, what are these functions?

In [ ]:
A = [1 2; 3 4]

@show maximum(A)
@show maximum(A, dims=1)
@show maximum(A, dims=2)

nothing

maximum(A) = 4
maximum(A, dims=1) = [3 4]
maximum(A, dims=2) = [2; 4]


In [ ]:
A = [1 2; 3 4]

@show minimum(A)
@show minimum(A, dims=1)
@show minimum(A, dims=2)

nothing

minimum(A) = 1
minimum(A, dims=1) = [1 2]
minimum(A, dims=2) = [1; 3]


Okay, fine, how would one add support to Flux then?

In [ ]:
# Forward
Base.maximum(xs::TrackedArray)         = track(maximum, xs)
Base.maximum(xs::TrackedArray, region) = track(maximum, xs, region)
Base.minimum(xs::TrackedArray)         = track(minimum, xs)
Base.minimum(xs::TrackedArray, region) = track(minimum, xs, region)

Now, what would the backwards pass look like?

In [ ]:
function back(::typeof(maximum), Δ, xs::TrackedArray)
    Δ′    = zeros(xs.data)
    _, i = findmax(xs.data)
    Δ′[i] = Δ
    @back(xs, Δ′)
end
function back(::typeof(maximum), Δ, xs::TrackedArray, region)
    Δ′     = zeros(xs.data)
    _, is  = findmax(xs.data, region)
    Δ′[is] = Δ
    @back(xs, Δ′)
end
function back(::typeof(minimum), Δ, xs::TrackedArray)
    Δ′    = zeros(xs.data)
    _, i  = findmin(xs.data)
    Δ′[i] = Δ
    @back(xs, Δ′)
end
function back(::typeof(minimum), Δ, xs::TrackedArray, region)
    Δ′     = zeros(xs.data)
    _, is  = findmin(xs.data, region)
    Δ′[is] = Δ
    @back(xs, Δ′)
end

There is a memory vs computation tradeoff made here, can you spot it?

This “patch” has now been in Flux for a few releases and `maximum` and `minimum` works as expected!

In [ ]:
A = param([1 2; 3 4])

@show maximum(A)
@show maximum(A, dims=1)
@show maximum(A, dims=2)

nothing

maximum(A) = 4.0 (tracked)
maximum(A, dims=1) = [3.0 4.0] (tracked)
maximum(A, dims=2) = [2.0; 4.0] (tracked)


So, what does a “real” model look like?

In [ ]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated

# Classify MNIST digits with a simple multi-layer-perceptron

imgs = MNIST.images()
# Stack images into one large batch
X = hcat(float.(reshape.(imgs, :))...)

labels = MNIST.labels()
# One-hot-encode the labels
Y = onehotbatch(labels, 0:9)

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax)

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

dataset = repeated((X, Y), 200)
evalcb = () -> @show(loss(X, Y))
opt = ADAM()

Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 10))

accuracy(X, Y)

# Test set accuracy
tX = hcat(float.(reshape.(MNIST.images(:test), :))...)
tY = onehotbatch(MNIST.labels(:test), 0:9)

accuracy(tX, tY)

┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/dkJUV/src/data/mnist.jl:24
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   469  100   469    0     0    590      0 --:--:-- --:--:-- --:--:--   589
100 9680k  100 9680k    0     0  2902k      0  0:00:03  0:00:03 --:--:-- 6038k
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/dkJUV/src/data/mnist.jl:24
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   469  100   469    0     0    622      0 --:--:-- --:--:-- --:--:--   622
100 28881  100 28881    0     0  16742      0  0:00:01  0:00:01 --:--:-- 50491
┌ Info: Downloading MNIST dataset
└ @ Flux.Data.MNIST /root/.julia/packages/Flux/dkJUV/src/data/mnist.jl:24
  % Total    % Received % Xferd  Average Spe

loss(X, Y) = 2.349295f0 (tracked)
loss(X, Y) = 2.3335562f0 (tracked)
loss(X, Y) = 2.3185475f0 (tracked)
loss(X, Y) = 2.3042197f0 (tracked)
loss(X, Y) = 2.2904954f0 (tracked)
loss(X, Y) = 2.2772849f0 (tracked)
loss(X, Y) = 2.2645047f0 (tracked)
loss(X, Y) = 2.252074f0 (tracked)
loss(X, Y) = 2.2399235f0 (tracked)
loss(X, Y) = 2.2280014f0 (tracked)
loss(X, Y) = 2.2162662f0 (tracked)
loss(X, Y) = 2.2046838f0 (tracked)
loss(X, Y) = 2.1932302f0 (tracked)
loss(X, Y) = 2.1818867f0 (tracked)
loss(X, Y) = 2.170641f0 (tracked)
loss(X, Y) = 2.159475f0 (tracked)
loss(X, Y) = 2.1483757f0 (tracked)
loss(X, Y) = 2.1373317f0 (tracked)
loss(X, Y) = 2.126334f0 (tracked)
loss(X, Y) = 2.1153736f0 (tracked)
loss(X, Y) = 2.1044407f0 (tracked)
loss(X, Y) = 2.0935295f0 (tracked)
loss(X, Y) = 2.0826368f0 (tracked)
loss(X, Y) = 2.0717576f0 (tracked)
loss(X, Y) = 2.060886f0 (tracked)
loss(X, Y) = 2.050013f0 (tracked)
loss(X, Y) = 2.0391347f0 (tracked)
loss(X, Y) = 2.0282464f0 (tracked)
loss(X, Y) = 2.0173416f0 (t

0.4576

## Extracurricular reading ##

1. [“On Machine Learning and Programming Languages”](https://julialang.org/blog/2017/12/ml&pl) by Innes et al. (2017)
2. [“Why Swift for TensorFlow”](https://github.com/tensorflow/swift/blob/master/docs/WhySwiftForTensorFlow.md) by Lattner et al. (2018)
3. [“A Differentiable Programming System to Bridge Machine Learning and Scientific Computing”](https://arxiv.org/abs/1907.07587) by Innes et al. (2019)
3. [“Differentiable Programming Manifesto”](https://github.com/apple/swift/blob/master/docs/DifferentiableProgramming.md) by Wei et al. (2019)

## Acknowledgements ##

A big thank you to James BRADBURY, a member of the Swift for TensorFlow team at Google AI – also, a Julia fanboy – for many useful discussions on these matters.